# Baseline for BERT: Naive Bayes + (Tf-idf or Count) 

In [148]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score 
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

import pandas as pd
import glob
import os
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


## Load data

Load the labeled dataset from all available decades.

In [190]:
DATA_DIR = '../../data/merged_split/'
dir_1960 = os.path.join(DATA_DIR, "1960s")
dir_1970 = os.path.join(DATA_DIR, "1970s")
dir_1980 = os.path.join(DATA_DIR, "1980s")
dir_1990 = os.path.join(DATA_DIR, "1990s")

def read_data(data_dir):
    df = pd.concat(map(pd.read_csv,glob.glob(data_dir+"/*.csv")))
    return df[['text_split','labels']]
    
df_1960 = read_data(dir_1960)
df_1970 = read_data(dir_1970)
df_1980 = read_data(dir_1980)
df_1990 = read_data(dir_1990)

dfs_lst = [df_1960,df_1970,df_1980,df_1990]
dfs = pd.concat(dfs_lst)

# dfs['labels'].value_counts()
# df_1960.loc[df_1960['labels']==2,'labels']=1
# print(df_1960['labels'].value_counts())



## Split dataset

Keep 20% of the dataset for the final test.

In [191]:
X = dfs.text_split.values
y = dfs.labels.values


X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.2, random_state=2022)


## Preprocessing

In [192]:
stopword_dutch = set(stopwords.words("dutch"))


In [193]:
#wordnet_lemmatizer = WordNetLemmatizer()
dutchStemmer=SnowballStemmer("dutch")

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Isolate and remove punctuations
    - Remove other special characters
    - Steming
    - Remove trailing whitespace
    """
    
    s = s.lower()
    # Isolate and remove punctuations
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', ' ', s)
    #s = re.sub(r'[^\w\s]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    
    words = nltk.word_tokenize(s)
    # Remove stopwords except 'not' and 'can'
    words = [word for word in words if word not in stopword_dutch]
    
    # Steming
    #s = " ".join([wordnet_lemmatizer.lemmatize(word, pos='v') for word in  words])
    s = " ".join([dutchStemmer.stem(word) for word in  words])

    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

In [194]:
# Preprocess text
X_train = np.array([text_preprocessing(text) for text in X_train])
X_val = np.array([text_preprocessing(text) for text in X_val])

## Parameters search
Apply hyperparameter optimization on CountVectorizer and TfidfVectorizer.

### Count Vectorizer

In [151]:
text_cnt_mnb = Pipeline([
    ("cnt", CountVectorizer()),
    ("clf",  MultinomialNB())
])

parameters = {
    'cnt__max_df': (0.5, 0.75, 1.0),
    'cnt__max_features': (None, 5000, 10000, 20000),
    'cnt__ngram_range': ((1, 1), (1, 2), (1,3)),
    'clf__alpha': (0.00001, 0.000001),
}

gs_cnt = GridSearchCV(text_cnt_mnb, parameters, cv=5, n_jobs=1, verbose=10)

In [152]:
gs_cnt = gs_cnt.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5; 1/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 1/5; 1/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.599 total time=   0.4s
[CV 2/5; 1/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 2/5; 1/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.598 total time=   0.4s
[CV 3/5; 1/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 3/5; 1/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.600 total time=   0.4s
[CV 4/5; 1/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 4/5; 1/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.587 t

[CV 5/5; 7/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 1);, score=0.582 total time=   0.4s
[CV 1/5; 8/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 1/5; 8/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2);, score=0.546 total time=   1.1s
[CV 2/5; 8/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 2/5; 8/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2);, score=0.555 total time=   1.1s
[CV 3/5; 8/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 3/5; 8/72] END clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2);, score=0.555 total time=   1.1s
[CV 4/5; 8/72] START clf__alpha=1e-05, cnt__max_df=0.5, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 4/5; 8/72] END clf__alpha=1e-05,

[CV 4/5; 14/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.603 total time=   1.1s
[CV 5/5; 14/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 2)
[CV 5/5; 14/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.610 total time=   1.1s
[CV 1/5; 15/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 3)
[CV 1/5; 15/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 3);, score=0.606 total time=   2.2s
[CV 2/5; 15/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 3)
[CV 2/5; 15/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 3);, score=0.611 total time=   2.4s
[CV 3/5; 15/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 3)
[CV 3/5; 15/72] END clf__alp

[CV 3/5; 21/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.552 total time=   2.3s
[CV 4/5; 21/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 4/5; 21/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.539 total time=   2.6s
[CV 5/5; 21/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 5/5; 21/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.554 total time=   2.2s
[CV 1/5; 22/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=20000, cnt__ngram_range=(1, 1)
[CV 1/5; 22/72] END clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=20000, cnt__ngram_range=(1, 1);, score=0.589 total time=   0.4s
[CV 2/5; 22/72] START clf__alpha=1e-05, cnt__max_df=0.75, cnt__max_features=20000, cnt__ngram_range=(1, 1)
[CV 2/5; 22/72] END 

[CV 2/5; 28/72] END clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1);, score=0.563 total time=   0.4s
[CV 3/5; 28/72] START clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1)
[CV 3/5; 28/72] END clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1);, score=0.563 total time=   0.4s
[CV 4/5; 28/72] START clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1)
[CV 4/5; 28/72] END clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1);, score=0.555 total time=   0.4s
[CV 5/5; 28/72] START clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1)
[CV 5/5; 28/72] END clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 1);, score=0.565 total time=   0.4s
[CV 1/5; 29/72] START clf__alpha=1e-05, cnt__max_df=1.0, cnt__max_features=5000, cnt__ngram_range=(1, 2)
[CV 1/5; 29/72] END clf__alpha=1e-05

...

[CV 5/5; 41/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 2);, score=0.552 total time=   1.1s
[CV 1/5; 42/72] START clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3)
[CV 1/5; 42/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3);, score=0.526 total time=   2.2s
[CV 2/5; 42/72] START clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3)
[CV 2/5; 42/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3);, score=0.531 total time=   2.3s
[CV 3/5; 42/72] START clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3)
[CV 3/5; 42/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3);, score=0.535 total time=   2.2s
[CV 4/5; 42/72] START clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=5000, cnt__ngram_range=(1, 3)
[CV 4/5; 42/72] END clf__alpha=1e-06

[CV 4/5; 48/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=20000, cnt__ngram_range=(1, 3);, score=0.556 total time=   2.2s
[CV 5/5; 48/72] START clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=20000, cnt__ngram_range=(1, 3)
[CV 5/5; 48/72] END clf__alpha=1e-06, cnt__max_df=0.5, cnt__max_features=20000, cnt__ngram_range=(1, 3);, score=0.562 total time=   2.1s
[CV 1/5; 49/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 1/5; 49/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.599 total time=   0.4s
[CV 2/5; 49/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 2/5; 49/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 1);, score=0.598 total time=   0.4s
[CV 3/5; 49/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=None, cnt__ngram_range=(1, 1)
[CV 3/5; 49/72] END clf__alp

[CV 3/5; 55/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 1);, score=0.580 total time=   0.4s
[CV 4/5; 55/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 1)
[CV 4/5; 55/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 1);, score=0.567 total time=   0.4s
[CV 5/5; 55/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 1)
[CV 5/5; 55/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 1);, score=0.583 total time=   0.4s
[CV 1/5; 56/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 1/5; 56/72] END clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 2);, score=0.546 total time=   1.1s
[CV 2/5; 56/72] START clf__alpha=1e-06, cnt__max_df=0.75, cnt__max_features=10000, cnt__ngram_range=(1, 2)
[CV 2/5; 56/72] END 

[CV 2/5; 62/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.610 total time=   1.1s
[CV 3/5; 62/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2)
[CV 3/5; 62/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.613 total time=   1.1s
[CV 4/5; 62/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2)
[CV 4/5; 62/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.602 total time=   1.1s
[CV 5/5; 62/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2)
[CV 5/5; 62/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 2);, score=0.610 total time=   1.1s
[CV 1/5; 63/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=None, cnt__ngram_range=(1, 3)
[CV 1/5; 63/72] END clf__alpha=1e-06

[CV 1/5; 69/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.538 total time=   2.2s
[CV 2/5; 69/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 2/5; 69/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.552 total time=   2.3s
[CV 3/5; 69/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 3/5; 69/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.550 total time=   2.2s
[CV 4/5; 69/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 4/5; 69/72] END clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3);, score=0.540 total time=   2.2s
[CV 5/5; 69/72] START clf__alpha=1e-06, cnt__max_df=1.0, cnt__max_features=10000, cnt__ngram_range=(1, 3)
[CV 5/5; 69/72] END clf__alp

In [155]:
print(gs_cnt.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_cnt.best_params_[param_name]))

0.6088338045860283
clf__alpha: 1e-05
cnt__max_df: 1.0
cnt__max_features: None
cnt__ngram_range: (1, 2)


### Tfidf Vectorizer

In [156]:
text_tf_mnb = Pipeline([
    ("tf-idf", TfidfVectorizer()),
    ("clf",  MultinomialNB())
])
parameters_tfidf = {
    'tf-idf__use_idf': (True, False),
    'tf-idf__norm': ('l1', 'l2', None),
    'tf-idf__max_df': (0.5, 0.75, 1.0),
    'tf-idf__max_features': (None, 5000, 10000, 20000),
    'tf-idf__ngram_range': ((1, 1), (1, 2), (1,3)),
    'clf__alpha': (0.00001, 0.000001),
}

gs_tf = GridSearchCV(text_tf_mnb, parameters_tfidf, cv=5, n_jobs=1, verbose=10)

In [157]:
gs_tf = gs_tf.fit(X_train, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5; 1/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=True
[CV 1/5; 1/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=True;, score=0.600 total time=   1.1s
[CV 2/5; 1/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=True
[CV 2/5; 1/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=True;, score=0.605 total time=   0.4s
[CV 3/5; 1/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=True
[CV 3/5; 1/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 

[CV 5/5; 5/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.586 total time=   0.4s
[CV 1/5; 6/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=False
[CV 1/5; 6/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=False;, score=0.599 total time=   0.4s
[CV 2/5; 6/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=False
[CV 2/5; 6/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=False;, score=0.598 total time=   0.4s
[CV 3/5; 6/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 1), tf-idf__norm=

[CV 5/5; 10/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False;, score=0.614 total time=   1.1s
[CV 1/5; 11/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm=None, tf-idf__use_idf=True
[CV 1/5; 11/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.602 total time=   1.2s
[CV 2/5; 11/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm=None, tf-idf__use_idf=True
[CV 2/5; 11/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.608 total time=   1.2s
[CV 3/5; 11/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 2), tf-idf__norm

[CV 4/5; 15/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True;, score=0.606 total time=   2.4s
[CV 5/5; 15/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True
[CV 5/5; 15/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True;, score=0.611 total time=   2.3s
[CV 1/5; 16/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=False
[CV 1/5; 16/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=False;, score=0.612 total time=   2.3s
[CV 2/5; 16/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=None, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf

[CV 3/5; 20/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=False;, score=0.564 total time=   0.4s
[CV 4/5; 20/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=False
[CV 4/5; 20/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=False;, score=0.551 total time=   0.4s
[CV 5/5; 20/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=False
[CV 5/5; 20/432] END clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l1, tf-idf__use_idf=False;, score=0.565 total time=   0.4s
[CV 1/5; 21/432] START clf__alpha=1e-05, tf-idf__max_df=0.5, tf-idf__max_features=5000, tf-idf__ngram_range=(1, 1), tf-idf__norm=l2,

...

[CV 4/5; 409/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=True;, score=0.557 total time=   2.2s
[CV 5/5; 409/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=True
[CV 5/5; 409/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=True;, score=0.563 total time=   2.2s
[CV 1/5; 410/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=False
[CV 1/5; 410/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=False;, score=0.549 total time=   2.2s
[CV 2/5; 410/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf_

[CV 3/5; 414/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=None, tf-idf__use_idf=False;, score=0.550 total time=   2.2s
[CV 4/5; 414/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=None, tf-idf__use_idf=False
[CV 4/5; 414/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=None, tf-idf__use_idf=False;, score=0.540 total time=   2.2s
[CV 5/5; 414/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=None, tf-idf__use_idf=False
[CV 5/5; 414/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=10000, tf-idf__ngram_range=(1, 3), tf-idf__norm=None, tf-idf__use_idf=False;, score=0.552 total time=   2.2s
[CV 1/5; 415/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1

[CV 2/5; 419/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.578 total time=   0.4s
[CV 3/5; 419/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True
[CV 3/5; 419/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.577 total time=   0.4s
[CV 4/5; 419/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True
[CV 4/5; 419/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1), tf-idf__norm=None, tf-idf__use_idf=True;, score=0.567 total time=   0.4s
[CV 5/5; 419/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 1),

[CV 1/5; 424/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False;, score=0.578 total time=   1.1s
[CV 2/5; 424/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False
[CV 2/5; 424/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False;, score=0.587 total time=   1.2s
[CV 3/5; 424/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False
[CV 3/5; 424/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-idf__norm=l2, tf-idf__use_idf=False;, score=0.587 total time=   1.1s
[CV 4/5; 424/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 2), tf-i

[CV 5/5; 428/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l1, tf-idf__use_idf=False;, score=0.570 total time=   2.3s
[CV 1/5; 429/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True
[CV 1/5; 429/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True;, score=0.576 total time=   2.3s
[CV 2/5; 429/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True
[CV 2/5; 429/432] END clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__norm=l2, tf-idf__use_idf=True;, score=0.582 total time=   2.3s
[CV 3/5; 429/432] START clf__alpha=1e-06, tf-idf__max_df=1.0, tf-idf__max_features=20000, tf-idf__ngram_range=(1, 3), tf-idf__

In [159]:
print(gs_tf.best_score_)
for param_name in sorted(parameters_tfidf.keys()):
    print("%s: %r" % (param_name, gs_tf.best_params_[param_name]))

0.6166356926944525
clf__alpha: 1e-05
tf-idf__max_df: 0.5
tf-idf__max_features: None
tf-idf__ngram_range: (1, 3)
tf-idf__norm: 'l1'
tf-idf__use_idf: False


## Model

Create the best model with the parameters collected in the previous section.

In [183]:
model_tfidf = make_pipeline(TfidfVectorizer(max_df = 0.5, max_features=None, ngram_range = (1, 3), norm = 'l1', use_idf = True), MultinomialNB(alpha = 1e-06))
model_count = make_pipeline(CountVectorizer(max_df = 1.0, max_features=None, ngram_range = (1, 2)), MultinomialNB(alpha = 1e-05))

In [184]:
model_tfidf.fit(X_train, y_train)


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.5, ngram_range=(1, 3), norm='l1')),
                ('multinomialnb', MultinomialNB(alpha=1e-06))])

In [185]:
model_count.fit(X_train, y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('multinomialnb', MultinomialNB(alpha=1e-05))])

In [186]:
y_pred_tfidf = model_tfidf.predict(X_val)
y_pred_count = model_count.predict(X_val)

## Prediction

In [195]:
print('Multinomial Naive Bayes with TF-IDF:')
print('-' * 40)
print(confusion_matrix(y_val, y_pred_tfidf))
print(classification_report(y_val, y_pred_tfidf, labels=[0,1,2]))

Multinomial Naive Bayes with TF-IDF:
----------------------------------------
[[2163  161  871]
 [ 401  447  604]
 [ 688  274 2338]]
              precision    recall  f1-score   support

           0       0.67      0.68      0.67      3195
           1       0.51      0.31      0.38      1452
           2       0.61      0.71      0.66      3300

    accuracy                           0.62      7947
   macro avg       0.60      0.56      0.57      7947
weighted avg       0.61      0.62      0.61      7947



In [196]:
print('Multinomial Naive Bayes with Count:')
print('-' * 40)
print(confusion_matrix(y_val, y_pred_count))
print(classification_report(y_val, y_pred_count, labels=[0,1,2]))

Multinomial Naive Bayes with Count:
----------------------------------------
[[2077  322  796]
 [ 309  671  472]
 [ 665  457 2178]]
              precision    recall  f1-score   support

           0       0.68      0.65      0.67      3195
           1       0.46      0.46      0.46      1452
           2       0.63      0.66      0.65      3300

    accuracy                           0.62      7947
   macro avg       0.59      0.59      0.59      7947
weighted avg       0.62      0.62      0.62      7947



## 2-Classes  vs 3-Classes

Compare the accuracy of the model on two datasets (3-class: negative, nutral, and positve vs 2-class: negative, non-negative)

In [200]:
dfs_2 = dfs.copy()
dfs_2.loc[dfs_2['labels']==2,'labels']=1
dfs_2['labels'].value_counts()


1    23836
0    15898
Name: labels, dtype: int64

In [201]:
X2 = dfs_2.text_split.values
y2 = dfs_2.labels.values


X_train2, X_val2, y_train2, y_val2 =\
    train_test_split(X2, y2, test_size=0.2, random_state=2022)



In [203]:
model_tfidf.fit(X_train2, y_train2)


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_df=0.5, ngram_range=(1, 3), norm='l1')),
                ('multinomialnb', MultinomialNB(alpha=1e-06))])

In [204]:
model_count.fit(X_train2, y_train2)

Pipeline(steps=[('countvectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('multinomialnb', MultinomialNB(alpha=1e-05))])

In [206]:
y_pred_tfidf2 = model_tfidf.predict(X_val2)
y_pred_count2 = model_count.predict(X_val2)

In [207]:
print('Multinomial Naive Bayes with TF-IDF- 2 classes:')
print('-' * 40)
print(confusion_matrix(y_val2, y_pred_tfidf2))
print(classification_report(y_val2, y_pred_tfidf2, labels=[0,1]))

Multinomial Naive Bayes with TF-IDF- 2 classes:
----------------------------------------
[[1715 1480]
 [ 640 4112]]
              precision    recall  f1-score   support

           0       0.73      0.54      0.62      3195
           1       0.74      0.87      0.80      4752

    accuracy                           0.73      7947
   macro avg       0.73      0.70      0.71      7947
weighted avg       0.73      0.73      0.72      7947



In [208]:
print('Multinomial Naive Bayes with Count- 2 classes:')
print('-' * 40)
print(confusion_matrix(y_val2, y_pred_count2))
print(classification_report(y_val2, y_pred_count2, labels=[0,1]))

Multinomial Naive Bayes with Count- 2 classes:
----------------------------------------
[[2015 1180]
 [ 954 3798]]
              precision    recall  f1-score   support

           0       0.68      0.63      0.65      3195
           1       0.76      0.80      0.78      4752

    accuracy                           0.73      7947
   macro avg       0.72      0.71      0.72      7947
weighted avg       0.73      0.73      0.73      7947



## Conclusion
The best models are as follows:

    - Dataset with 3-classes: **Multinomial Naive Bayes with Count with accuracy : 0.62, macro avg : 0.59**
    
    - Dataset with 2-classes: **Multinomial Naive Bayes with Count with accuracy : 0.73, macro avg : 0.71**